<a href="https://colab.research.google.com/github/nyanta012/chatgpt_api_practice/blob/main/section7/section7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colaboratoryのショートカットキー
1. **現在のセルを実行し、次のセルを選択**: `Shift + Enter`
2. **コードセルをテキストセルにする**: `Ctrl + M M`
3. **コードセルを上に追加**: `Ctrl + M A`
4. **コードセルを下に追加**: `Ctrl + M B`
5. **セルの削除**: `Ctrl + M D`

# API KEYの設定

In [ ]:
%%capture
!pip install openai==0.28.1

In [ ]:
import openai
import getpass
import json
from IPython.display import Markdown, display


apikey = getpass.getpass(prompt="OpenAIのAPIキーを入力してください")
openai.api_key = apikey

OpenAIのAPIキーを入力してください··········


**Function Callingとは･･･**
　簡単に言うと、ユーザーの入力文から外部の関数を利用できる形式(JSON形式)に変換する機能

In [ ]:
# 使いたい外部の関数を定義
def get_current_weather(location, unit):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": 25,
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [ ]:
# 与えた関数の説明を記載
my_functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use. Infer this from the users location.",
                },
            },
            "required": ["location", "unit"],
        },
    }
]

In [ ]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[{"role": "user", "content": "東京の天気は何でしょうか?"}],
    functions=my_functions,
    function_call="auto",
)

In [ ]:
response["choices"][0]["message"].get("function_call")

<OpenAIObject at 0x7ffa56939080> JSON: {
  "name": "get_current_weather",
  "arguments": "{\n  \"location\": \"Tokyo\",\n  \"unit\": \"celsius\"\n}"
}

In [ ]:
response["choices"][0]["message"]["function_call"]["arguments"]

'{\n  "location": "Tokyo",\n  "unit": "celsius"\n}'

In [ ]:
json_response = json.loads(response["choices"][0]["message"]["function_call"]["arguments"])

In [ ]:
json_response["unit"]

'celsius'

In [ ]:
get_current_weather(location=json_response["location"], unit=json_response["unit"])

'{"location": "Tokyo", "temperature": 25, "unit": "celsius", "forecast": ["sunny", "windy"]}'

In [ ]:
function_response = get_current_weather(location=json_response["location"], unit=json_response["unit"])

In [ ]:
function_name = response["choices"][0]["message"]["function_call"]["name"]

In [ ]:
message = response["choices"][0]["message"]

In [ ]:
# 関数の結果を使って回答を生成させる
second_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "user", "content": "東京の天気は何でしょうか?"},
        message,
        {
            "role": "function",
            "name": function_name,
            "content": function_response,
        },
    ],
)


In [ ]:
print(second_response["choices"][0]["message"]["content"])

東京の天気は晴れで、気温は摂氏25度です。また、風も強めです。


関数が不要な質問をしてみると･･･

In [ ]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[{"role": "user", "content": "猫を英語で言うと何ですか？"}],
    functions=my_functions,
    function_call="auto",
)

In [ ]:
response["choices"][0]["message"].get("function_call")

In [ ]:
response["choices"][0]["message"]["content"]

'猫を英語で言うと"cat"です。'

強制的に使用させることもできる

In [ ]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[{"role": "user", "content": "猫を英語で言うと何ですか？"}],
    functions=my_functions,
    function_call={"name":"get_current_weather"},
)

In [ ]:
response["choices"][0]["message"].get("function_call")

<OpenAIObject at 0x7ffa40e0efc0> JSON: {
  "name": "get_current_weather",
  "arguments": "{\n  \"location\": \"San Francisco, CA\",\n  \"unit\": \"celsius\"\n}"
}

In [ ]:
json.loads(response["choices"][0]["message"]["function_call"]["arguments"])

{'location': 'San Francisco, CA', 'unit': 'celsius'}

## まとめ

1. ユーザーの入力内容を基に、GPTが関数の説明(`my_functions`)を読み、関数を使うべきか判断する。

2. 判断結果によるアクション：
    - **使わなくて良い場合**：通常通り回答生成を行う
    - **使うべき場合**：その関数を使うために必要な引数を考える

3. 必要な引数をJSON形式の文字列で出力する

**注意**：②のステップでは、強制的に関数を使わせることも可能。


# Function CallingでCSVから回答を生成させてみよう

In [ ]:
import pandas as pd

In [ ]:
json_list = {
    "商品A": {
        "特徴": "革新的なデザインが特徴の商品Aです。スリムなボディで、持ち運びも容易です。また、高い耐久性を持ち、長期間使用することが可能です。さらに、エコフレンドリーな材料を使用して製造されています。"
    },
    "商品B": {
        "特徴": "商品Bは独自のテクノロジーにより、効率的な性能を発揮します。充電一回で長時間使用することができ、忙しい日々にピッタリです。さらに、直感的な操作性を誇り、誰でも簡単に使いこなすことができます。"
    },
    "商品C": {
        "特徴": "商品Cは高い解像度と鮮やかな色彩表現が魅力的です。映画鑑賞やゲームに最適で、臨場感あふれる体験を提供します。また、スタイリッシュなデザインは、どの部屋にもマッチします。"
    },
    "商品D": {
        "特徴": "商品Dは、飛躍的に進化したバッテリーライフを誇ります。持ち運びに便利なサイズ感で、外出先でも使用可能です。さらに、強力な防水性能も兼ね備えています。"
    },
    "商品E": {
        "特徴": "商品Eはユーザーフレンドリーなインターフェースが特徴です。初心者でも容易に操作が可能で、簡単に学べます。さらに、高いカスタマイズ性を持ち、ユーザーのニーズに応じた使い方が可能です。持続的なソフトウェアアップデートも提供しています。"
    },
    "商品F": {
        "特徴": "商品Fは、独自の音響技術により、高品質な音を提供します。どんな音楽も深みと広がりを持って再現し、まるでライブ会場にいるかのような感覚を味わえます。軽量でコンパクトなデザインも特徴で、持ち運びも容易です。"
    },
    "商品G": {
        "特徴": "商品Gは、エネルギー効率の良さが特徴です。高いパフォーマンスを維持しつつ、省電力化を実現しています。それにより、長時間の使用でも電力を大幅に節約できます。また、冷却機能も優れており、過熱を防ぎます。"
    },
    "商品H": {
        "特徴": "商品Hは、精密な作業が可能な高性能なツールです。多機能でありながら直感的な操作性を持ち、様々な作業をスムーズにこなします。耐久性の高い素材で作られており、長期間の使用に耐える設計となっています。"
    },
    "商品I": {
        "特徴": "商品Iは、画期的なクリーニング機能が特徴です。自動的に汚れを検出し、最適なクリーニング方法を選択します。その結果、ユーザーは最小限の労力で清潔な状態を維持することができます。さらに、静音設計により騒音を抑えています。"
    },
    "商品J": {
        "特徴": "商品Jは、プロフェッショナルな仕上がりを実現する高品質な材料を使用しています。細部までこだわったデザインと、優れた耐久性が特徴です。さらに、使い心地を追求した形状と質感が、高い満足度を提供します。"
    },
    "商品K": {
        "特徴": "商品Kは、業界をリードするパフォーマンスを発揮します。最新の技術を搭載し、一般的な製品よりも優れた処理速度と効率性を実現します。さらに、信頼性と安定性の高さが、長期間の使用に対応します。"
    },
    "商品L": {
        "特徴": "商品Lは、健康をサポートする高精度なセンサーを搭載しています。リアルタイムで体調管理が可能で、早期の健康問題を予防します。また、使いやすいアプリと連携し、個々のニーズに対応するカスタマイズが可能です。"
    },
    "商品M": {
        "特徴": "商品Mは、高度な通信技術により、迅速かつ安定した接続性能を実現します。データの送受信がスムーズで、オンラインでの作業やゲームに最適です。また、セキュリティ機能も充実しており、安全な通信環境を提供します。"
    },
    "商品N": {
        "特徴": "商品Nは、シンプルかつ洗練されたデザインが特徴です。どんなインテリアにも溶け込み、お部屋の雰囲気を引き立てます。さらに、快適性を追求した設計により、長時間の使用でも快適さを維持します。"
    },
    "商品O": {
        "特徴": "商品Oは、高精度な計測能力を持つ優れたデバイスです。一般的な製品よりも正確なデータを提供し、より信頼性の高い結果を得られます。使いやすいインターフェースと連携して、計測結果の管理と分析を容易にします。"
    },
    "商品P": {
        "特徴": "商品Pは、創新的な3Dプリンティング技術を利用しています。自由度の高いデザインと高精度な造形が可能で、製造業から教育まで幅広い用途に対応します。また、使いやすいソフトウェアと連搐して、誰でも簡単に使用できます。"
    },
    "商品Q": {
        "特徴": "商品Qは、最先端のAI技術を駆使した製品です。ユーザーの行動を学習し、最適な提案を行います。その結果、日々の生活がより便利で効率的になります。さらに、プライバシー保護機能も充実しており、安心して使用できます。"
    },
    "商品R": {
        "特徴": "商品Rは、高速かつ効率的な充電技術が特徴です。一晩でフル充電が可能で、持ち運びにも適しています。また、安全性を確保するための過充電防止機能も搭載されています。"
    },
    "商品S": {
        "特徴": "商品Sは、究極の快適性を追求したアイテムです。最高級の素材と独自の技術で、長時間の使用でも快適さを維持します。さらに、洗練されたデザインが、スタイリッシュなライフスタイルを演出します。"
    },
    "商品T": {
        "特徴": "商品Tは、優れた保温・保冷性能を持つアイテムです。長時間にわたり飲料や食品の温度を保つことができます。また、軽量かつコンパクトなデザインで、持ち運びも容易です。耐久性も高く、長期間使用可能です。"
    },
    "商品U": {
        "特徴": "商品Uは、独特の美しいデザインと優れた機能性を兼ね備えたアイテムです。使用時には高いパフォーマンスを発揮し、使用しない時には室内を美しく彩ります。また、環境に配慮したエコフレンドリーな素材を使用しています。"
    },
    "商品V": {
        "特徴": "商品Vは、精密なセンサー技術により高精度な分析が可能です。実世界の情報をデジタル化し、深い洞察を提供します。それにより、より知識を深めたり、意思決定を行うことが可能となります。さらに、セキュリティ機能も充実しております。"
    },
    "商品W": {
        "特徴": "商品Wは、軽量で強力なエンジン性能が特徴です。加速や旋回性能が優れており、スポーティな走行を楽しむことができます。また、最新の安全装備も搭載しており、安心して乗ることができます。"
    },
    "商品X": {
        "特徴": "商品Xは、立体的な音響体験を提供するヘッドセットです。最新のサラウンド技術により、まるで現場にいるかのような音響を再現します。また、長時間の使用にも適した快適な装着感と、充電持続時間が特徴です。"
    },
    "商品Y": {
        "特徴": "商品Yは、生活の様々なシーンで活躍する多機能ガジェットです。コンパクトながらも高性能な部品を組み込み、一つで複数の機能を果たします。持ち運びも容易で、旅行や出張などにも最適です。"
    },
    "商品Z": {
        "特徴": "商品Zは、高速なデータ転送能力が特徴です。複数のデバイスとの同期もスムーズで、効率的なデジタルライフをサポートします。さらに、セキュリティ機能も充実しており、安心してご使用いただけます。"
    }
}

In [ ]:
"""
json_listをそのまま使用しても良いが、
ここではExcelやcsvなどのデータを扱う場合を想定
"""
df = pd.DataFrame(json_list)
df = df.T.reset_index()
df.columns = ["商品名", "特徴"]

In [ ]:
dict_description = dict(zip(df["商品名"].values, df["特徴"].values))

In [ ]:
def get_product_description(product_name):
    """
    商品名から特徴を返す関数
    """
    description = dict_description.get(product_name)
    if not description:
        description = "該当データがありません"
    return description

In [ ]:
def get_chatgpt_response_with_description(user_input):
    # 与えた関数の説明を記載
    my_functions = [
        {
            "name": "get_product_description",
            "description": "商品名から特徴を返す関数",
            "parameters": {
                "type": "object",
                "properties": {
                    "product_name": {
                        "type": "string",
                        "description": "商品名を抽出する。例:商品A",
                    },
                },
                "required": ["product_name"],
            },
        }
    ]

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": user_input}],
        functions=my_functions,
        function_call={"name": "get_product_description"},
    )

    message = response["choices"][0]["message"]

    # 今回は必ず使われるので無くても良い
    if message.get("function_call"):
        function_name = message["function_call"]["name"]
        function_args = json.loads(message["function_call"]["arguments"])

        function_response = get_product_description(
            product_name=function_args.get("product_name"),
        )

        # 関数を使った結果を含めて回答を生成する
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[
                {"role": "user", "content": user_input},
                message,
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                },
            ],
        )

        return second_response["choices"][0]["message"]["content"]

In [ ]:
response = get_chatgpt_response_with_description("商品Bについて教えてください")

In [ ]:
print(response)

商品Bは独自のテクノロジーを搭載しており、高い性能を発揮します。一回の充電で長時間使用することができ、忙しい日常生活に最適です。操作も直感的で、誰でも簡単に使いこなすことができます。


In [ ]:
response = get_chatgpt_response_with_description("商品Dについて教えてください")
print(response)

商品Dは、持ち運びに便利なサイズ感でありながら、飛躍的に進化したバッテリーライフを誇る商品です。これにより、外出先でも長時間の使用が可能となります。また、商品Dは強力な防水性能を持っており、雨や水のかかる場面でも安心して使用することができます。商品Dは、ユーザーにとって非常に便利で信頼性の高い製品となっています。


In [ ]:
response = get_chatgpt_response_with_description("商品Wについて、中学生にもわかるように説明してください")
print(response)

商品Wは、すごくパワフルで軽くて、速く走ることができる車です。曲がるときもすごくスムーズで、スポーツカーのような感じで楽しむことができるよ！そして、最新の安全装置もついていて、安心して乗ることができますよ！


In [ ]:
response = get_chatgpt_response_with_description("商品ABBについて教えてください")
print(response)

商品ABBについて詳細な情報を提供することができません。具体的な商品名や説明があれば、再度お知らせいただければ幸いです。


In [ ]:
df

,商品名,特徴
0,商品A,革新的なデザインが特徴の商品Aです。スリムなボディで、持ち運びも容易です。また、高い耐久性を...
1,商品B,商品Bは独自のテクノロジーにより、効率的な性能を発揮します。充電一回で長時間使用することがで...
2,商品C,商品Cは高い解像度と鮮やかな色彩表現が魅力的です。映画鑑賞やゲームに最適で、臨場感あふれる体...
3,商品D,商品Dは、飛躍的に進化したバッテリーライフを誇ります。持ち運びに便利なサイズ感で、外出先でも...
4,商品E,商品Eはユーザーフレンドリーなインターフェースが特徴です。初心者でも容易に操作が可能で、簡単...
5,商品F,商品Fは、独自の音響技術により、高品質な音を提供します。どんな音楽も深みと広がりを持って再現...
6,商品G,商品Gは、エネルギー効率の良さが特徴です。高いパフォーマンスを維持しつつ、省電力化を実現して...
7,商品H,商品Hは、精密な作業が可能な高性能なツールです。多機能でありながら直感的な操作性を持ち、様々...
8,商品I,商品Iは、画期的なクリーニング機能が特徴です。自動的に汚れを検出し、最適なクリーニング方法を...
9,商品J,商品Jは、プロフェッショナルな仕上がりを実現する高品質な材料を使用しています。細部までこだわ...
